# CS 224D Assignment #2
# Part [2]: Recurrent Neural Networks

This notebook will provide starter code, testing snippets, and additional guidance for implementing the Recurrent Neural Network Language Model (RNNLM) described in Part 2 of the handout.

Please complete parts (a), (b), and (c) of Part 2 before beginning this notebook.

In [2]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## (e): Implement a Recurrent Neural Network Language Model

Follow the instructions on the handout to implement your model in `rnnlm.py`, then use the code below to test.

In [3]:
from rnnlm import RNNLM
# Gradient check on toy data, for speed
random.seed(10)
wv_dummy = random.randn(10,50)
model = RNNLM(L0 = wv_dummy, U0 = wv_dummy,
              alpha=0.005, rseed=10, bptt=4)
model.grad_check(array([1,2,3]), array([2,3,4]))

NOTE: temporarily setting self.bptt = len(y) = 3 to compute true gradient.
grad_check: dJ/dH error norm = 7.731e-10 [ok]
    H dims: [50, 50] = 2500 elem
grad_check: dJ/dU error norm = 9.496e-10 [ok]
    U dims: [10, 50] = 500 elem
grad_check: dJ/dL[1] error norm = 4.411e-10 [ok]
    L[1] dims: [50] = 50 elem
grad_check: dJ/dL[3] error norm = 4.461e-10 [ok]
    L[3] dims: [50] = 50 elem
grad_check: dJ/dL[2] error norm = 4.076e-10 [ok]
    L[2] dims: [50] = 50 elem
Reset self.bptt = 4


## Prepare Vocabulary and Load PTB Data

We've pre-prepared a list of the vocabulary in the Penn Treebank, along with their absolute counts and unigram frequencies. The document loader code below will "canonicalize" words and replace any unknowns with a `"UUUNKKK"` token, then convert the data to lists of indices.

In [4]:
from data_utils import utils as du
import pandas as pd

# Load the vocabulary
vocab = pd.read_table("data/lm/vocab.ptb.txt", header=None, sep="\s+",
                     index_col=0, names=['count', 'freq'], )

# Choose how many top words to keep
vocabsize = 2000
num_to_word = dict(enumerate(vocab.index[:vocabsize]))
word_to_num = du.invert_dict(num_to_word)
##
# Below needed for 'adj_loss': DO NOT CHANGE
fraction_lost = float(sum([vocab['count'][word] for word in vocab.index
                           if (not word in word_to_num) 
                               and (not word == "UUUNKKK")]))
fraction_lost /= sum([vocab['count'][word] for word in vocab.index
                      if (not word == "UUUNKKK")])
print "Retained %d words from %d (%.02f%% of all tokens)" % (vocabsize, len(vocab),
                                                             100*(1-fraction_lost))

Retained 2000 words from 38444 (84.00% of all tokens)


Load the datasets, using the vocabulary in `word_to_num`. Our starter code handles this for you, and also generates lists of lists X and Y, corresponding to input words and target words*. 

*(Of course, the target words are just the input words, shifted by one position, but it can be cleaner and less error-prone to keep them separate.)*

In [5]:
# Load the training set
docs = du.load_dataset('data/lm/ptb-train.txt')
S_train = du.docs_to_indices(docs, word_to_num)
X_train, Y_train = du.seqs_to_lmXY(S_train)

# Load the dev set (for tuning hyperparameters)
docs = du.load_dataset('data/lm/ptb-dev.txt')
S_dev = du.docs_to_indices(docs, word_to_num)
X_dev, Y_dev = du.seqs_to_lmXY(S_dev)

# Load the test set (final evaluation only)
docs = du.load_dataset('data/lm/ptb-test.txt')
S_test = du.docs_to_indices(docs, word_to_num)
X_test, Y_test = du.seqs_to_lmXY(S_test)

# Display some sample data
print " ".join(d[0] for d in docs[7])
print S_test[7]

Big investment banks refused to step up to the plate to support the beleaguered floor traders by buying big blocks of stock , traders say .
[   4  147  169  250 1879    7 1224   64    7    1    3    7  456    1    3
 1024  255   24  378  147    3    6   67    0  255  138    2    5]


## (f): Train and evaluate your model

When you're able to pass the gradient check, let's run our model on some real language!

You should randomly initialize the word vectors as Gaussian noise, i.e. $W_{ij} \sim \mathit{N}(0,0.1)$; the function `random.randn` may be helpful here.

In [6]:
hdim = 100 # dimension of hidden layer = dimension of word vectors
random.seed(10)
L0 = zeros((vocabsize, hdim)) # replace with random init, 
                              # or do in RNNLM.__init__()
model = RNNLM(L0, U0 = L0, alpha=0.1, rseed=10, bptt=1)

# Gradient check is going to take a *long* time here
# since it's quadratic-time in the number of parameters.
# run at your own risk...
# model.grad_check(array([1,2,3]), array([2,3,4]))

In [ ]:
#### YOUR CODE HERE ####

##
# Pare down to a smaller dataset, for speed (optional)
ntrain = len(Y_train)
X = X_train[:ntrain]
Y = Y_train[:ntrain]

k = 20
indices = range(ntrain)

# A random schedule of N/k minibatches of size k, sampled with replacement from the training set.
def idxiter_batches():
    num_batches = ntrain / k
    for i in xrange(num_batches):
        yield random.choice(indices, k)

model_output = model.train_sgd(X=X, y=Y, idxiter=None, printevery=1, costevery=10000)

#### END YOUR CODE ####

Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.43107
  Seen 1 in 511.60 s
  Seen 2 in 511.62 s
  Seen 3 in 511.66 s
  Seen 4 in 511.72 s
  Seen 5 in 511.75 s
  Seen 6 in 511.77 s
  Seen 7 in 511.79 s
  Seen 8 in 511.81 s
  Seen 9 in 511.86 s
  Seen 10 in 511.88 s
  Seen 11 in 511.91 s
  Seen 12 in 511.92 s
  Seen 13 in 511.93 s
  Seen 14 in 511.94 s
  Seen 15 in 511.96 s
  Seen 16 in 512.00 s
  Seen 17 in 512.04 s
  Seen 18 in 512.09 s
  Seen 19 in 512.12 s
  Seen 20 in 512.12 s
  Seen 21 in 512.15 s
  Seen 22 in 512.16 s
  Seen 23 in 512.18 s
  Seen 24 in 512.22 s
  Seen 25 in 512.24 s
  Seen 26 in 512.25 s
  Seen 27 in 512.31 s
  Seen 28 in 512.34 s
  Seen 29 in 512.35 s
  Seen 30 in 512.40 s
  Seen 31 in 512.42 s
  Seen 32 in 512.45 s
  Seen 33 in 512.48 s
  Seen 34 in 512.51 s
  Seen 35 in 512.54 s
  Seen 36 in 512.57 s
  Seen 37 in 512.58 s
  Seen 38 in 512.60 s
  Seen 39 in 512.62 s
  Seen 40 in 512.67 s
  Seen 41 in 512.71 s
  Seen 42 in 512.75 s
  Seen 43 in 512.78 s
  Seen

In [9]:
## Evaluate cross-entropy loss on the dev set,
## then convert to perplexity for your writeup
dev_loss = model.compute_mean_loss(X_dev, Y_dev)

In [13]:
print dev_loss

4.13280675333


The performance of the model is skewed somewhat by the large number of `UUUNKKK` tokens; if these are 1/6 of the dataset, then that's a sizeable fraction that we're just waving our hands at. Naively, our model gets credit for these that's not really deserved; the formula below roughly removes this contribution from the average loss. Don't worry about how it's derived, but do report both scores - it helps us compare across models with different vocabulary sizes.

In [10]:
## DO NOT CHANGE THIS CELL ##
# Report your numbers, after computing dev_loss above.
def adjust_loss(loss, funk):
    return (loss + funk * log(funk))/(1 - funk)
print "Unadjusted: %.03f" % exp(dev_loss)
print "Adjusted for missing vocab: %.03f" % exp(adjust_loss(dev_loss, fraction_lost))

Unadjusted: 62.353
Adjusted for missing vocab: 96.622


### Save Model Parameters

In [11]:
##
# Save to .npy files; should only be a few MB total
assert(min(model.sparams.L.shape) <= 100) # don't be too big
assert(max(model.sparams.L.shape) <= 5000) # don't be too big
save("rnnlm.L.npy", model.sparams.L)
save("rnnlm.U.npy", model.params.U)
save("rnnlm.H.npy", model.params.H)

## (g): Generating Data

Once you've trained your model to satisfaction, let's use it to generate some sentences!

Implement the `generate_sequence` function in `rnnlm.py`, and call it below.

In [12]:
def seq_to_words(seq):
    return [num_to_word[s] for s in seq]
    
seq, J = model.generate_sequence(word_to_num["<s>"], 
                                 word_to_num["</s>"], 
                                 maxlen=100)
print J
# print seq
print " ".join(seq_to_words(seq))

86.6578173188
<s> the people has be relatively in cash in UUUNKKK in UUUNKKK and UUUNKKK for the bear under or DGDG . </s>


**BONUS:** Use the unigram distribution given in the `vocab` table to fill in any `UUUNKKK` tokens in your generated sequences with words that we omitted from the vocabulary. You'll want to use `list(vocab.index)` to get a list of words, and `vocab.freq` to get a list of corresponding frequencies.

In [17]:
# Replace UUUNKKK with a random unigram,
# drawn from vocab that we skipped
from nn.math import MultinomialSampler, multinomial_sample
def fill_unknowns(words):
    #### YOUR CODE HERE ####
    ret = words # do nothing; replace this
    

    #### END YOUR CODE ####
    return ret
    
print " ".join(fill_unknowns(seq_to_words(seq)))